# Pandas + dates + times

Mirrored at: https://github.com/reuven/SFPython-2021-08Aug-18

# Agenda

1. Date/time concepts and data structures
2. Timestamp columns (how to read them)
3. Time delta columns (what they are, and how to work with them)
4. Date/time indexes on data frames
5. Time series and resampling


# Time and date concepts

There are basically two of them:

- One moment in time (when the meeting starts, or when you were born) -- timestamp/datetime, representing one specific moment
- A span of time (how long the meeting goes, or how long you have been alive) -- timedelta -- doesn't have a specific start or end, but rather is a length of time

In [1]:
import pandas as pd
from pandas import Series, DataFrame

In [2]:
df = pd.read_csv('taxi.csv')

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,40.754093,2,17.0,0.0,0.5,0.00,0.0,0.3,17.80
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,40.761909,1,6.5,0.0,0.5,1.00,0.0,0.3,8.30
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,40.745438,1,8.0,0.0,0.5,2.20,0.0,0.3,11.00
3,2,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,N,-73.971527,40.760330,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
4,1,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,N,-73.982162,40.758999,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30


In [4]:
df = pd.read_csv('taxi.csv',
                usecols=['tpep_pickup_datetime', 'tpep_dropoff_datetime',
                        'trip_distance',
                        'passenger_count',
                        'total_amount'])

In [5]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,17.80
1,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,8.30
2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,11.00
3,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,17.16
4,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,10.30


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tpep_pickup_datetime   9999 non-null   object 
 1   tpep_dropoff_datetime  9999 non-null   object 
 2   passenger_count        9999 non-null   int64  
 3   trip_distance          9999 non-null   float64
 4   total_amount           9999 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 390.7+ KB


In [9]:
df = pd.read_csv('taxi.csv',
                usecols=['tpep_pickup_datetime', 'tpep_dropoff_datetime',
                        'trip_distance',
                        'passenger_count',
                        'total_amount'],
                 parse_dates=['tpep_pickup_datetime',
                             'tpep_dropoff_datetime']
                )

In [10]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,17.80
1,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,8.30
2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,11.00
3,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,17.16
4,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,10.30


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tpep_pickup_datetime   9999 non-null   datetime64[ns]
 1   tpep_dropoff_datetime  9999 non-null   datetime64[ns]
 2   passenger_count        9999 non-null   int64         
 3   trip_distance          9999 non-null   float64       
 4   total_amount           9999 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(1)
memory usage: 390.7 KB


In [13]:
df['tpep_pickup_datetime'].dt.hour

0       11
1       11
2       11
3       11
4       11
        ..
9994     0
9995     0
9996     0
9997     0
9998     0
Name: tpep_pickup_datetime, Length: 9999, dtype: int64